# Notebook para ejecutar NSForest

Este notebook está preparado para ejecutar en GoogleColab por necesidades de RAM

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
!pip install scanpy
import scanpy as sc

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/TFM/src/NSForest/scripts/')

## Import libreries

In [4]:
from NSForest_v3dot9_2_test import *

## Load data

In [5]:
# Loading h5ad
file = "/content/drive/MyDrive/TFM/src/NSForest/data/adata_switched_2_2.h5ad"
adata = sc.read_h5ad(file)
adata_test = sc.read_h5ad("/content/drive/MyDrive/TFM/src/NSForest/data/adata_selected_park_6cl.h5ad")

## Quick view of datasets

In [6]:
adata #quick look of the data

AnnData object with n_obs × n_vars = 85429 × 16653
    obs: 'Origin', 'suspension_type', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'author_cell_type', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'cell_type_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'is_primary_data', 'development_stage_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'donor_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'batch'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'schema_version', 'title'
    obsm: 'X_scANVI', 'X_scVI', 'X_umap'

In [7]:
adata_test

AnnData object with n_obs × n_vars = 20571 × 16653
    obs: 'Origin', 'suspension_type', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'author_cell_type', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'cell_type_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'is_primary_data', 'development_stage_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'donor_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'schema_version', 'title'
    obsm: 'X_scANVI', 'X_scVI', 'X_umap'

## Get cluster names

In [8]:
cluster_header = "cell_type" #<---
set(adata.obs[cluster_header])

{'endothelial cell',
 'epithelial cell of proximal tubule',
 'kidney collecting duct principal cell',
 'kidney connecting tubule epithelial cell',
 'kidney distal convoluted tubule epithelial cell',
 'renal beta-intercalated cell'}

In [9]:
np.unique(adata.obs[cluster_header])

array(['endothelial cell', 'epithelial cell of proximal tubule',
       'kidney collecting duct principal cell',
       'kidney connecting tubule epithelial cell',
       'kidney distal convoluted tubule epithelial cell',
       'renal beta-intercalated cell'], dtype=object)

In [10]:
len(np.unique(adata.obs[cluster_header]))

6

In [11]:
# Extrae el nombre del archivo sin la extensión
dataset_name = os.path.splitext(os.path.basename(file))[0]
outputfilename = "RF"
output_folder = f"../outputs_experimentation/RF/{dataset_name}/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Creating new directory...\n{output_folder}")

Creating new directory...
../outputs_experimentation/RF/adata_switched_2_2/


## Runing NSmRMR to obtain the biomarkers

### Runing NSmRMR using all clusters

In [12]:
NSForest_results = NSForest(adata, cluster_header=cluster_header, n_trees=100, n_genes_eval=6, beta = 0.2,
                            output_folder = output_folder, outputfilename_prefix = outputfilename) #<---

Preparing data...
--- 6.199422121047974 seconds ---
Calculating medians...
--- 107.88094687461853 seconds ---
Number of clusters to evaluate: 6
1 out of 6:
	endothelial cell
Tiempo de ejecución: 380.92 segundos
	Only 9 out of 15 top Random Forest features with median > 0 will be further evaluated.
	['ENSMUSG00000021759', 'ENSMUSG00000039706']
	fbeta: 0.952259847093361
2 out of 6:
	renal beta-intercalated cell
Tiempo de ejecución: 299.45 segundos
	['ENSMUSG00000028238', 'ENSMUSG00000020566']
	fbeta: 0.9562839364274107
3 out of 6:
	epithelial cell of proximal tubule
Tiempo de ejecución: 268.90 segundos
	Only 5 out of 15 top Random Forest features with median > 0 will be further evaluated.
	['ENSMUSG00000030945', 'ENSMUSG00000076441', 'ENSMUSG00000055373']
	fbeta: 0.9288443657958668
4 out of 6:
	kidney connecting tubule epithelial cell
Tiempo de ejecución: 257.81 segundos
	['ENSMUSG00000054640', 'ENSMUSG00000028030']
	fbeta: 0.8254410897195259
5 out of 6:
	kidney distal convoluted tubule 

In [13]:
NSForest_results

,clusterName,clusterSize,f_score,PPV,TN,FP,FN,TP,marker_count,NSForest_markers,thresholds,binary_genes
0,endothelial cell,4824,0.952260,0.990658,80583,22,2491,2333,2,"[ENSMUSG00000021759, ENSMUSG00000039706]","[0.8481884002685547, 0.7238551378250122]","[ENSMUSG00000021759, ENSMUSG00000039706, ENSMU..."
1,renal beta-intercalated cell,1756,0.956284,0.988285,83662,11,828,928,2,"[ENSMUSG00000028238, ENSMUSG00000020566]","[0.4547166973352432, 0.4649316668510437]","[ENSMUSG00000061080, ENSMUSG00000028238, ENSMU..."
2,epithelial cell of proximal tubule,59680,0.928844,0.949790,23860,1889,23947,35733,3,"[ENSMUSG00000030945, ENSMUSG00000076441, ENSMU...","[0.40984900295734406, 0.31009024381637573, 0.1...","[ENSMUSG00000030945, ENSMUSG00000076441, ENSMU..."
3,kidney connecting tubule epithelial cell,3072,0.825441,0.950241,82326,31,2480,592,2,"[ENSMUSG00000054640, ENSMUSG00000028030]","[2.118518114089966, 1.3690828680992126]","[ENSMUSG00000054640, ENSMUSG00000031558, ENSMU..."
4,kidney distal convoluted tubule epithelial cell,11491,0.935723,0.966446,73730,208,5500,5991,2,"[ENSMUSG00000031766, ENSMUSG00000028017]","[1.1833228468894958, 1.0618106722831726]","[ENSMUSG00000031766, ENSMUSG00000022490, ENSMU..."
5,kidney collecting duct principal cell,4606,0.898290,0.954416,80743,80,2931,1675,2,"[ENSMUSG00000023013, ENSMUSG00000004988]","[1.4653971195220947, 1.1409903764724731]","[ENSMUSG00000023013, ENSMUSG00000004988, ENSMU..."
6,Average,14238,0.916140,0.966639,70817,373,6362,7875,-,-,-,-


### Testing the model performance

### Approach 1:
- Train a decission tree for each of selected genes as biomarkers by NSForest
- Obtein the prediction as the dot product of the individual prediction for each gene

In [14]:
NSForest_results = NSForest_results.iloc[:-1]
df_test_result = myDecisionTreeEvaluationTest(adata, adata_test, cluster_header, NSForest_results, beta = 0.2,
                                              output_folder = output_folder, outputfilename_prefix = outputfilename)

In [15]:
df_test_result

,clusterName,clusterSize,f_score,PPV,TN,FP,FN,TP,marker_count,NSForest_markers,threshold
0,endothelial cell,638,0.271147,1.000000,19933,0,629,9,2,"['ENSMUSG00000021759', 'ENSMUSG00000039706']","[0.8481884002685547, 0.7238551378250122]"
1,renal beta-intercalated cell,691,0.903678,0.973799,19874,6,468,223,2,"['ENSMUSG00000028238', 'ENSMUSG00000020566']","[0.4547166973352432, 0.4649316668510437]"
2,epithelial cell of proximal tubule,13152,0.911479,0.969361,7267,152,8343,4809,3,"['ENSMUSG00000030945', 'ENSMUSG00000076441', '...","[0.40984900295734406, 0.31009024381637573, 0.1..."
3,kidney connecting tubule epithelial cell,831,0.000000,0.000000,19740,0,831,0,2,"['ENSMUSG00000054640', 'ENSMUSG00000028030']","[2.118518114089966, 1.3690828680992126]"
4,kidney distal convoluted tubule epithelial cell,4890,0.911411,0.958373,15594,87,2887,2003,2,"['ENSMUSG00000031766', 'ENSMUSG00000028017']","[1.1833228468894958, 1.0618106722831726]"
5,kidney collecting duct principal cell,369,0.937311,0.957627,20192,10,143,226,2,"['ENSMUSG00000023013', 'ENSMUSG00000004988']","[1.4653971195220947, 1.1409903764724731]"
6,Average,3428,0.655838,0.809860,17100,42,2216,1211,-,-,-


### Approach 2:
- Train a decission tree with all NSmRMR selected gene at the same time

In [16]:
df_combined_test_result = myDecisionTreeEvaluationTestCombined(adata, adata_test, cluster_header, NSForest_results, beta = 0.2,
                                                      output_folder = output_folder, outputfilename_prefix = outputfilename)

In [17]:
df_combined_test_result

,clusterName,clusterSize,f_score,PPV,TN,FP,FN,TP,marker_count,NSForest_markers,threshold,NSAUCROC_markers
0,endothelial cell,638,0.662252,0.943396,19930,3,588,50,2,"['ENSMUSG00000021759', 'ENSMUSG00000039706']",[0.8481884002685547],NaN
1,renal beta-intercalated cell,691,0.707209,0.714815,19726,154,305,386,2,"['ENSMUSG00000028238', 'ENSMUSG00000020566']",[0.4547166973352432],NaN
2,epithelial cell of proximal tubule,13152,0.828174,0.832054,5450,1969,3397,9755,3,"['ENSMUSG00000030945', 'ENSMUSG00000076441', '...",[0.40984900295734406],NaN
3,kidney connecting tubule epithelial cell,831,0.405983,0.423469,19514,226,665,166,2,"['ENSMUSG00000054640', 'ENSMUSG00000028030']",[2.118518114089966],NaN
4,kidney distal convoluted tubule epithelial cell,4890,0.833702,0.845893,15135,546,1893,2997,2,"['ENSMUSG00000031766', 'ENSMUSG00000028017']",[1.1833228468894958],NaN
5,kidney collecting duct principal cell,369,0.711595,0.709924,20088,114,90,279,2,"['ENSMUSG00000023013', 'ENSMUSG00000004988']",[1.4653971195220947],NaN
6,Average,3428,0.691486,0.744925,16640,502,1156,2272,-,NaN,-,-


In [18]:
new_eval = myDecisionTreeEvaluationTestMove(adata, adata_test, cluster_header, NSForest_results, beta = 0.2,
                                                               output_folder = output_folder, outputfilename_prefix = outputfilename, coef=0.5)

In [19]:
new_eval

,clusterName,clusterSize,f_score,PPV,TN,FP,FN,TP,marker_count,NSForest_markers,original_threshold,new_threshold
0,endothelial cell,638,0.223862,1.000000,19933,0,631,7,2,"['ENSMUSG00000021759', 'ENSMUSG00000039706']","[0.8481884002685547, 0.7238551378250122]","[0.884131882339716, 0.7579623945057392]"
1,renal beta-intercalated cell,691,0.896081,0.980100,19876,4,494,197,2,"['ENSMUSG00000028238', 'ENSMUSG00000020566']","[0.4547166973352432, 0.4649316668510437]","[0.5515786409378052, 0.5622370913624763]"
2,epithelial cell of proximal tubule,13152,0.888683,0.977093,7335,84,9569,3583,3,"['ENSMUSG00000030945', 'ENSMUSG00000076441', '...","[0.40984900295734406, 0.31009024381637573, 0.1...","[0.824158564209938, 0.7408173680305481, 0.4810..."
3,kidney connecting tubule epithelial cell,831,0.000000,0.000000,19740,0,831,0,2,"['ENSMUSG00000054640', 'ENSMUSG00000028030']","[2.118518114089966, 1.3690828680992126]","[2.376619905233383, 1.4482608661055565]"
4,kidney distal convoluted tubule epithelial cell,4890,0.887445,0.976401,15649,32,3566,1324,2,"['ENSMUSG00000031766', 'ENSMUSG00000028017']","[1.1833228468894958, 1.0618106722831726]","[1.6282113194465637, 1.4157971143722534]"
5,kidney collecting duct principal cell,369,0.938741,0.963470,20194,8,158,211,2,"['ENSMUSG00000023013', 'ENSMUSG00000004988']","[1.4653971195220947, 1.1409903764724731]","[1.6408644318580627, 1.2931244373321533]"
6,Average,3428,0.639135,0.816177,17121,21,2541,887,-,-,-,-
